# InsightAI Data vector processing and making the vector database

# Package installation

In [1]:
!pip install -qU langchain transformers sentence-transformers datasets qdrant-client openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 92.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.9/258.9 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.9/362.9 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 91.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77

# Loading the dataset

**The Dataset originally contains more than 20k products from different fashion brands**
see the dataset [here](https://huggingface.co/datasets/Melikshah/products)

In [2]:
from datasets import load_dataset

fashion = load_dataset(
    "Melikshah/products",
    split="train"
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/20842 [00:00<?, ? examples/s]

**See the fields and number of rows**

In [3]:
fashion

Dataset({
    features: ['product_id', 'Description', 'Specifications', 'Gender', 'Image_links', 'Company', 'Category', 'Price', 'Link', 'Name'],
    num_rows: 20842
})

## Data Processing

In [4]:
product_df = fashion.to_pandas()
product_data = product_df.reset_index(drop=True).to_dict(orient="index")
product_df

,product_id,Description,Specifications,Gender,Image_links,Company,Category,Price,Link,Name
0,0,Charcoal black tie-dyed cotton coaty with blue...,Colour is Black. Fabric is Cotton. Value Addit...,Man,[https://www.aarong.com/media/catalog/product/...,Aarong,men coaty,"Tk 2,493.02",https://www.aarong.com/catalog/product/view/id...,Charcoal Black Tie-Dyed And Embroidered Cotton...
1,1,Slate blue cotton Taaga Man waistcoat. Feature...,Colour is Blue. Fabric is Cotton. Cut /Fit is ...,Man,[https://www.aarong.com/media/catalog/product/...,Aarong,men coaty,"Tk 2,218.60",https://www.aarong.com/catalog/product/view/id...,Slate Blue Cotton Taaga Man Waistcoat
2,2,Grey cotton Taaga Man waistcoat. Features five...,Colour is Grey. Fabric is Cotton. Cut /Fit is ...,Man,[https://www.aarong.com/media/catalog/product/...,Aarong,men coaty,"Tk 2,218.60",https://www.aarong.com/catalog/product/view/id...,Grey Cotton Taaga Man Waistcoat
3,3,Midnight blue and white textured cotton coaty....,Colour is Blue. Fabric is Cotton. Cut /Fit is ...,Man,[https://www.aarong.com/media/catalog/product/...,Aarong,men coaty,"Tk 1,269.77",https://www.aarong.com/catalog/product/view/id...,Midnight Blue Textured Cotton Coaty
4,4,Grey mixed cotton Taaga Man waistcoat. Feature...,Colour is Grey. Fabric is Mixed Cotton. Cut /F...,Man,[https://www.aarong.com/media/catalog/product/...,Aarong,men coaty,"Tk 2,916.28",https://www.aarong.com/catalog/product/view/id...,Grey Mixed Cotton Taaga Man Slim Fit Waistcoat
...,...,...,...,...,...,...,...,...,...,...
20837,20837,A woman's dress with a floral print and a matc...,Product Name: Orange Print Indian Carpe Fabric...,Woman,[https://infinitymegamall.com/wp-content/uploa...,Infinity,Woman FLAT 30%,"Tk 2,490.60",https://infinitymegamall.com/product/womens-si...,Women's Single Kameez
20838,20838,"The image features a woman wearing a long, flo...","The image features a woman wearing a long, flo...",Woman,[https://infinitymegamall.com/wp-content/uploa...,Infinity,Woman Flat 40%,"Tk 2,392.95",https://infinitymegamall.com/product/womens-si...,Women's Single Kameez
20839,20839,"The image features a mannequin wearing a long,...",Product Name: Chocolate Silk Dress\n\nFabric: ...,Woman,[https://infinitymegamall.com/wp-content/uploa...,Infinity,Woman Women,"Tk 3,467.10",https://infinitymegamall.com/product/arabian-c...,Arabian Classic Burqa
20840,20840,The image features a mannequin wearing a blue ...,Product Name: Blue Dress\n\nFabric: Cotton\n\n...,Woman,[https://infinitymegamall.com/wp-content/uploa...,Infinity,Woman Flat 40%,"Tk 2,392.95",https://infinitymegamall.com/product/womens-si...,Women's Single Kameez


In [5]:
columns_to_check = ['Name', 'Description', 'Specifications', 'Price']
for column in columns_to_check:
    product_df[column] = product_df[column].fillna("not mentioned")

# Environment File

In [7]:
import os
os.environ['QDRANT_HOST'] =    "******************************************************"
os.environ['QDRANT_API_KEY'] = "******************************************************"
os.environ['OPENAI_API_KEY'] = "******************************************************"

# Necessary Imports

In [9]:
import qdrant_client
import openai
from qdrant_client import QdrantClient
from qdrant_client.http import models
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
from qdrant_client.http.models import VectorParams, Distance

**Making the openai clients**

In [10]:
import openai
import qdrant_client

openai_client = openai.Client(
    api_key=os.getenv('OPENAI_API_KEY')
)

**Making Qdrant client**

In [11]:
qdrantClient = qdrant_client.QdrantClient(
        os.getenv("QDRANT_HOST"),
        api_key=os.getenv("QDRANT_API_KEY")
    )

**Helper to make the collection**

In [12]:
def make_collection(collection_name:str):
    try:
        qdrantClient.get_collection(collection_name)
        return "Collection Already exists"
    except Exception as e:
        if "Not found" in str(e):
            qdrantClient.create_collection(
                collection_name,
                vectors_config={
                    "summary": VectorParams(
                        size=3072,
                        distance=Distance.COSINE
                    ),
                    "image": VectorParams(
                        size=512,
                        distance=Distance.COSINE
                    )
                },
            )
            rt_str = "Collection named:" + collection_name + "has been successfully created"
            return rt_str
        else:
            return "Error occurred making the collection"

In [13]:
make_collection("InsightAI_shopgenie")

'Collection named:InsightAI_shopgeniehas been successfully created'

**Multimodal Model**

In [18]:
import torch
image_model = SentenceTransformer('clip-ViT-B-32', device='cuda' if torch.cuda.is_available() else 'cpu')

modules.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/118 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/1.91k [00:00<?, ?B/s]

0_CLIPModel/vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

0_CLIPModel/tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

...

0_CLIPModel/merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

0_CLIPModel/tokenizer_config.json:   0%|          | 0.00/733 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

0_CLIPModel/config.json:   0%|          | 0.00/4.54k [00:00<?, ?B/s]

0_CLIPModel/special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

**Helper to Generate the embeddings**

In [19]:
def create_embedding(txt):
    embedding_model = "text-embedding-3-large"
    str_embedding = openai_client.embeddings.create(input= txt, model=embedding_model)
    return str_embedding.data[0].embedding

***Base-64 image conversion Helper***

In [20]:
from PIL import Image
from io import BytesIO
import base64

def pil_image_to_base64(pil_image):
    """
    Convert a PIL image to a base64-encoded string.

    Args:
        pil_image (PIL.JpegImagePlugin.JpegImageFile): A PIL image object.

    Returns:
        str: The base64-encoded image data.
    """
    byte_io = BytesIO()
    pil_image.save(byte_io, format='JPEG')
    byte_image = byte_io.getvalue()
    base64_image = base64.b64encode(byte_image).decode('utf-8')
    return base64_image

**Inserting the points in vector Database**

In [ ]:
import uuid
import requests
from pathlib import Path

output_directory = Path("./imagesall")
output_directory.mkdir(exist_ok=True)


iteration_cnt = 0

for index, row in product_df.iterrows():
    iteration_cnt += 1

    document_id = str(uuid.uuid4())


    summary_str = (
        "This is a " + row['Category'] + " from " + row['Company'] + ". It is a " + row['Name'] + ". " +
        row['Description'] + ". Specifications: " + row['Specifications']
    )
    print(summary_str)

    summary_embedding = create_embedding(summary_str)

    # Handle image download and embedding
    image_url = row['Image_links'][0]
    image_filename = image_url.split('/')[-1].split('?')[0]
    image_path = output_directory / image_filename

    # Download the image if it doesn't already exist
    if not image_path.exists():
        response = requests.get(image_url)
        if response.status_code == 200:
            with open(image_path, 'wb') as f:
                f.write(response.content)
        else:
            print(f"Failed to download image from {image_url}")
            continue
    else:
        print(f"Image already exists at {image_path}")

    # Generate image embedding
    img = Image.open(image_path)
    query_embedding = image_model.encode(img, convert_to_tensor=False).tolist()

    # Upsert the data into the Qdrant collection
    qdrantClient.upsert(
        collection_name="InsightAI_shopgenie",
        points=[
            {
                "id": document_id,
                "vector": {
                    "summary": summary_embedding,
                    "image": query_embedding
                },
                "payload": {
                    "product_id": row["product_id"],
                    "Link": row["Link"],
                    "Name": row["Name"],
                    "Description": row["Description"],
                    "Price": row["Price"],
                    "Specifications": row["Specifications"],
                    "Image_links": row["Image_links"].tolist(),
                    "Gender": row["Gender"],
                    "Category": row["Category"],
                    "Company": row["Company"],
                }
            }
        ]
    )
    print(row["product_id"], " done")
    # break